In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import tensorflow as tf
from generate.vae_ import VariationalAutoEncoder
from generate.recons_utils import get_reconstructed_df
from generate.corr_loss_ import correlation_loss
from sklearn.preprocessing import MinMaxScaler
import pandas as pd

In [2]:
path = "amatis/33float/"
original_df = pd.read_csv(path+"B_Preprocessed.csv", ";")

/home/murat/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3398: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
original_df

,CROSSING,FINISSHING,HEADING_ACCURACY,SHORT_PASSING,VOLLEYS,DRIBBLING,CURVE,FREE_KICK_ACCURACY,LONG_PASSING,BALL_CONTROL,...,VISION,PENALTIES,MARKING,STANDING_TACKLE,SLIDING_TACKLE,GK_DIVING,GK_HANDLING,GK_KICKING,GK_POSITIONING,GK_REFLEXES
0,56.0,28.0,54.0,68.0,55.0,58.0,58.0,49.0,65.0,62.0,...,59.0,62.0,35.0,51.0,51.0,11.0,5.0,14.0,7.0,7.0
1,21.0,20.0,26.0,27.0,21.0,30.0,8.0,11.0,75.0,34.0,...,60.0,49.0,26.0,24.0,45.0,78.0,74.0,75.0,73.0,79.0
2,49.0,67.0,47.0,73.0,59.0,71.0,63.0,62.0,67.0,72.0,...,72.0,68.0,39.0,45.0,37.0,13.0,12.0,10.0,11.0,9.0
3,41.0,62.0,60.0,60.0,52.0,70.0,49.0,38.0,48.0,68.0,...,55.0,41.0,22.0,22.0,25.0,9.0,11.0,11.0,13.0,14.0
4,60.0,41.0,46.0,70.0,60.0,62.0,47.0,31.0,68.0,67.0,...,66.0,66.0,59.0,67.0,67.0,15.0,8.0,13.0,5.0,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11995,82.0,70.0,55.0,81.0,69.0,86.0,85.0,77.0,79.0,85.0,...,77.0,72.0,24.0,27.0,23.0,15.0,10.0,8.0,13.0,16.0
11996,61.0,52.0,66.0,74.0,67.0,66.0,45.0,67.0,71.0,72.0,...,65.0,66.0,52.0,68.0,68.0,6.0,5.0,6.0,14.0,10.0
11997,66.0,85.0,81.0,57.0,76.0,77.0,69.0,50.0,32.0,76.0,...,66.0,68.0,24.0,41.0,34.0,14.0,20.0,32.0,20.0,20.0
11998,69.0,55.0,60.0,75.0,54.0,66.0,53.0,59.0,72.0,71.0,...,69.0,61.0,61.0,66.0,64.0,15.0,5.0,9.0,13.0,14.0


In [4]:
ORIGINAL_DIM = 33
LATENT_DIM = 16
EPOCHS = 100
BATCH_SIZE = 128
COLS = original_df.columns

vae = VariationalAutoEncoder(
    original_dim = ORIGINAL_DIM,
    latent_dim = LATENT_DIM)

optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
mse_loss_fn = tf.keras.losses.MeanSquaredError()
cos_loss_fn = tf.keras.losses.CosineSimilarity()
# corr_loss_fn = correlation_loss
loss_metric = tf.keras.metrics.Mean()

In [5]:
todo -1 +1 scaling ile sigmoid (0, 1 ) fail

SyntaxError: invalid syntax (3771317148.py, line 1)

In [5]:
scaler = MinMaxScaler((-1, 1))
scaled_original_df = scaler.fit_transform(original_df)
df_tensor = tf.convert_to_tensor(scaled_original_df, dtype=tf.float32)

train_dataset = tf.data.Dataset.from_tensor_slices(df_tensor)
train_dataset_batch = train_dataset.shuffle(buffer_size=1024).batch(BATCH_SIZE)

In [6]:
# Iterate over epochs.
for epoch in range(EPOCHS):
    print("_"*70)
    print("Start of epoch %d" % (epoch,))
    # Iterate over the batches of the dataset.
    for step, x_batch_train in enumerate(train_dataset_batch):
        with tf.GradientTape() as tape:
            reconstructed = vae(x_batch_train)
            # Compute reconstruction loss
            loss = mse_loss_fn(x_batch_train, reconstructed)
            loss += -1*(correlation_loss(x_batch_train, reconstructed))
            loss += cos_loss_fn(x_batch_train, reconstructed)
            loss += sum(vae.losses) # Add KLD regularization loss

        grads = tape.gradient(loss, vae.trainable_weights)
        optimizer.apply_gradients(zip(grads, vae.trainable_weights))
        loss_metric(loss)
        if step % 30 == 0:
            print("step %d: mean loss = %.4f" % (step, loss_metric.result()))

______________________________________________________________________
Start of epoch 0
step 0: mean loss = 0.4449
step 30: mean loss = -0.9376
step 60: mean loss = -1.0541
step 90: mean loss = -1.1408
______________________________________________________________________
Start of epoch 1
step 0: mean loss = -1.1527
step 30: mean loss = -1.2334
step 60: mean loss = -1.2863
step 90: mean loss = -1.3249
______________________________________________________________________
Start of epoch 2
step 0: mean loss = -1.3289
step 30: mean loss = -1.3565
step 60: mean loss = -1.3770
step 90: mean loss = -1.3948
______________________________________________________________________
Start of epoch 3
step 0: mean loss = -1.3968
step 30: mean loss = -1.4116
step 60: mean loss = -1.4240
step 90: mean loss = -1.4354
______________________________________________________________________
Start of epoch 4
step 0: mean loss = -1.4367
step 30: mean loss = -1.4466
step 60: mean loss = -1.4557
step 90: mean l

step 0: mean loss = -1.5970
step 30: mean loss = -1.5972
step 60: mean loss = -1.5974
step 90: mean loss = -1.5976
______________________________________________________________________
Start of epoch 41
step 0: mean loss = -1.5976
step 30: mean loss = -1.5978
step 60: mean loss = -1.5980
step 90: mean loss = -1.5982
______________________________________________________________________
Start of epoch 42
step 0: mean loss = -1.5982
step 30: mean loss = -1.5984
step 60: mean loss = -1.5986
step 90: mean loss = -1.5988
______________________________________________________________________
Start of epoch 43
step 0: mean loss = -1.5988
step 30: mean loss = -1.5990
step 60: mean loss = -1.5992
step 90: mean loss = -1.5994
______________________________________________________________________
Start of epoch 44
step 0: mean loss = -1.5994
step 30: mean loss = -1.5996
step 60: mean loss = -1.5997
step 90: mean loss = -1.5999
_____________________________________________________________________

KeyboardInterrupt: 

In [7]:
# Iterate over epochs.
for epoch in range(EPOCHS):
    print("_"*70)
    print("Start of epoch %d" % (epoch,))
    # Iterate over the batches of the dataset.
    for step, x_batch_train in enumerate(train_dataset_batch):
        with tf.GradientTape() as tape:
            reconstructed = vae(x_batch_train)
            # Compute reconstruction loss
            loss = mse_loss_fn(x_batch_train, reconstructed)
            loss += -1*(correlation_loss(x_batch_train, reconstructed))
            loss += cos_loss_fn(x_batch_train, reconstructed)
            loss += sum(vae.losses) # Add KLD regularization loss

        grads = tape.gradient(loss, vae.trainable_weights)
        optimizer.apply_gradients(zip(grads, vae.trainable_weights))
        loss_metric(loss)
        if step % 30 == 0:
            print("step %d: mean loss = %.4f" % (step, loss_metric.result()))

______________________________________________________________________
Start of epoch 0
step 0: mean loss = 0.3548
step 30: mean loss = -0.3691
step 60: mean loss = -0.5588
step 90: mean loss = -0.6540
______________________________________________________________________
Start of epoch 1
step 0: mean loss = -0.6662
step 30: mean loss = -0.7328
step 60: mean loss = -0.7758
step 90: mean loss = -0.8082
______________________________________________________________________
Start of epoch 2
step 0: mean loss = -0.8122
step 30: mean loss = -0.8367
step 60: mean loss = -0.8540
step 90: mean loss = -0.8697
______________________________________________________________________
Start of epoch 3
step 0: mean loss = -0.8715
step 30: mean loss = -0.8843
step 60: mean loss = -0.8951
step 90: mean loss = -0.9046
______________________________________________________________________
Start of epoch 4
step 0: mean loss = -0.9060
step 30: mean loss = -0.9151
step 60: mean loss = -0.9220
step 90: mean l

step 30: mean loss = -1.0472
step 60: mean loss = -1.0477
step 90: mean loss = -1.0482
______________________________________________________________________
Start of epoch 41
step 0: mean loss = -1.0483
step 30: mean loss = -1.0488
step 60: mean loss = -1.0493
step 90: mean loss = -1.0497
______________________________________________________________________
Start of epoch 42
step 0: mean loss = -1.0498
step 30: mean loss = -1.0503
step 60: mean loss = -1.0507
step 90: mean loss = -1.0512
______________________________________________________________________
Start of epoch 43
step 0: mean loss = -1.0512
step 30: mean loss = -1.0517
step 60: mean loss = -1.0522
step 90: mean loss = -1.0526
______________________________________________________________________
Start of epoch 44
step 0: mean loss = -1.0527
step 30: mean loss = -1.0531
step 60: mean loss = -1.0535
step 90: mean loss = -1.0540
______________________________________________________________________
Start of epoch 45
step 0: 

step 90: mean loss = -1.0894
______________________________________________________________________
Start of epoch 81
step 0: mean loss = -1.0894
step 30: mean loss = -1.0897
step 60: mean loss = -1.0899
step 90: mean loss = -1.0901
______________________________________________________________________
Start of epoch 82
step 0: mean loss = -1.0902
step 30: mean loss = -1.0904
step 60: mean loss = -1.0906
step 90: mean loss = -1.0908
______________________________________________________________________
Start of epoch 83
step 0: mean loss = -1.0909
step 30: mean loss = -1.0911
step 60: mean loss = -1.0913
step 90: mean loss = -1.0915
______________________________________________________________________
Start of epoch 84
step 0: mean loss = -1.0915
step 30: mean loss = -1.0918
step 60: mean loss = -1.0920
step 90: mean loss = -1.0922
______________________________________________________________________
Start of epoch 85
step 0: mean loss = -1.0922
step 30: mean loss = -1.0925
step 60: 

In [7]:
reconstructed_df = get_reconstructed_df(vae, scaler, train_dataset_batch, COLS)
reconstructed_df

,CROSSING,FINISSHING,HEADING_ACCURACY,SHORT_PASSING,VOLLEYS,DRIBBLING,CURVE,FREE_KICK_ACCURACY,LONG_PASSING,BALL_CONTROL,...,VISION,PENALTIES,MARKING,STANDING_TACKLE,SLIDING_TACKLE,GK_DIVING,GK_HANDLING,GK_KICKING,GK_POSITIONING,GK_REFLEXES
0,65.790314,64.430069,58.180489,69.949860,62.935329,71.042114,66.365112,60.622608,61.389935,72.527657,...,69.805862,65.522896,36.558865,39.816605,38.834721,7.070482,18.153910,53.108780,18.752550,17.634695
1,52.127857,32.054173,70.235191,64.332886,33.564980,51.809456,40.953671,38.462460,59.829716,62.893707,...,51.246025,48.174267,72.330780,74.627449,72.940010,7.925040,9.323875,13.709140,9.246280,9.878754
2,61.201859,49.397903,73.979538,74.012947,53.831539,60.631451,60.339352,60.600456,72.358421,71.099030,...,66.412697,60.906773,73.968971,78.716400,74.534798,9.119357,8.192962,9.061569,8.577682,8.571648
3,43.797688,29.306700,73.574554,61.657398,31.073853,43.107563,37.489388,35.387665,57.279022,58.460236,...,49.608330,46.701519,73.560631,75.060448,72.197090,9.432058,11.672292,17.724447,12.062715,12.088832
4,65.828674,46.895905,62.638142,71.253441,49.185036,64.725639,60.306938,57.003437,66.403458,71.050072,...,62.812176,56.550396,67.344376,71.212395,69.404465,8.194138,9.912978,14.048297,10.101331,9.716454
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11995,21.103212,18.065384,28.343513,32.857010,16.451170,21.223555,17.962635,18.371967,50.251030,29.833683,...,42.259411,42.188667,28.207039,29.540236,27.025608,66.847755,64.314484,62.241997,64.599663,69.378113
11996,41.902203,54.507317,53.696308,52.038960,48.820759,54.838646,44.157482,39.260090,41.867058,55.964958,...,48.618347,51.588760,23.603067,27.179644,25.061172,17.683954,18.037943,18.251003,19.093016,18.395695
11997,13.402033,12.565114,11.817952,23.391996,12.543243,12.919923,13.495809,13.404945,29.694635,19.450109,...,29.993645,23.555740,11.588096,11.791448,11.537252,84.699623,79.945251,77.445503,81.560272,86.262558
11998,55.665165,35.208187,63.132538,63.956944,36.312763,53.790287,46.122337,41.504715,57.607883,62.659660,...,53.552261,47.986568,67.811371,70.206413,68.670746,8.598926,10.966606,17.239697,11.402437,11.240366


In [8]:
reconstructed_df.round()

,CROSSING,FINISSHING,HEADING_ACCURACY,SHORT_PASSING,VOLLEYS,DRIBBLING,CURVE,FREE_KICK_ACCURACY,LONG_PASSING,BALL_CONTROL,...,VISION,PENALTIES,MARKING,STANDING_TACKLE,SLIDING_TACKLE,GK_DIVING,GK_HANDLING,GK_KICKING,GK_POSITIONING,GK_REFLEXES
0,66.0,64.0,58.0,70.0,63.0,71.0,66.0,61.0,61.0,73.0,...,70.0,66.0,37.0,40.0,39.0,7.0,18.0,53.0,19.0,18.0
1,52.0,32.0,70.0,64.0,34.0,52.0,41.0,38.0,60.0,63.0,...,51.0,48.0,72.0,75.0,73.0,8.0,9.0,14.0,9.0,10.0
2,61.0,49.0,74.0,74.0,54.0,61.0,60.0,61.0,72.0,71.0,...,66.0,61.0,74.0,79.0,75.0,9.0,8.0,9.0,9.0,9.0
3,44.0,29.0,74.0,62.0,31.0,43.0,37.0,35.0,57.0,58.0,...,50.0,47.0,74.0,75.0,72.0,9.0,12.0,18.0,12.0,12.0
4,66.0,47.0,63.0,71.0,49.0,65.0,60.0,57.0,66.0,71.0,...,63.0,57.0,67.0,71.0,69.0,8.0,10.0,14.0,10.0,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11995,21.0,18.0,28.0,33.0,16.0,21.0,18.0,18.0,50.0,30.0,...,42.0,42.0,28.0,30.0,27.0,67.0,64.0,62.0,65.0,69.0
11996,42.0,55.0,54.0,52.0,49.0,55.0,44.0,39.0,42.0,56.0,...,49.0,52.0,24.0,27.0,25.0,18.0,18.0,18.0,19.0,18.0
11997,13.0,13.0,12.0,23.0,13.0,13.0,13.0,13.0,30.0,19.0,...,30.0,24.0,12.0,12.0,12.0,85.0,80.0,77.0,82.0,86.0
11998,56.0,35.0,63.0,64.0,36.0,54.0,46.0,42.0,58.0,63.0,...,54.0,48.0,68.0,70.0,69.0,9.0,11.0,17.0,11.0,11.0


In [9]:
reconstructed_df.round().to_csv(path+"C_SyntheticData.csv", sep=";", index=False)

In [10]:
from d_evaluation_metrics.main import evaluate
from e_report_generation.main import generate_report

# file_path = "amatis/train/train.csv"
file_path = "amatis/33float/33float.csv"

result_status, result_message = evaluate(file_path)
print("MODULE D -> ", result_message, result_status)

07/12/2022 05:28:41 - Main - INFO - ----------------------------------------------------------------------
07/12/2022 05:28:41 - Main - INFO - SECTION D - ANALYSIS HAS BEEN STARTED.
07/12/2022 05:28:41 - Main - INFO - ----------------------------------------------------------------------
07/12/2022 05:28:41 - ConfigOps - INFO - CONFIG OPERATIONS HAS BEEN STARTED.

07/12/2022 05:28:41 - ConfigOps - INFO - CONFIG OPERATIONS HAS BEEN COMPLETED.

07/12/2022 05:28:41 - DataFrameOps - INFO - DATAFRAME OPERATIONS HAS BEEN STARTED.

07/12/2022 05:28:41 - DataFrameOps - INFO - amatis/33float/B_Preprocessed.csv dataset file path has been validated.
07/12/2022 05:28:41 - DataFrameOps - INFO - amatis/33float/B_Preprocessed.csv dataset file has been loaded.
07/12/2022 05:28:41 - DataFrameOps - INFO - amatis/33float/C_SyntheticData.csv dataset file path has been validated.
07/12/2022 05:28:41 - DataFrameOps - INFO - amatis/33float/C_SyntheticData.csv dataset file has been loaded.
07/12/2022 05:28:41

amatis/33float/analysis.log
amatis/33float/analysis.log
amatis/33float/analysis.log
amatis/33float/analysis.log


07/12/2022 05:28:41 - VisualizationItems - INFO - CORRELATION MATRICES HAS BEEN COMPLETED.



COMMON COLS: ['SHORT_PASSING', 'FINISSHING', 'POSITIONING', 'ACCELERATION', 'DRIBBLING', 'CROSSING', 'SPRINT_SPEED', 'STANDING_TACKLE', 'AGGRESSION', 'GK_POSITIONING', 'HEADING_ACCURACY', 'AGILITY', 'STAMINA', 'GK_KICKING', 'MARKING', 'FREE_KICK_ACCURACY', 'JUMPING', 'GK_HANDLING', 'SLIDING_TACKLE', 'LONG_PASSING', 'GK_DIVING', 'STRENGTH', 'INTERCEPTIONS', 'PENALTIES', 'VISION', 'LONG_SHOTS', 'GK_REFLEXES', 'REACTION', 'VOLLEYS', 'BALL_CONTROL', 'CURVE', 'BALANCE', 'SHOT_POWER']


07/12/2022 05:28:41 - VisualizationItems - INFO - PCA REDUCTIONS HAS BEEN COMPLETED.

07/12/2022 05:28:41 - DuplicateCalculator - INFO - PRIVACY SCORING HAS BEEN STARTED.

07/12/2022 05:28:41 - DuplicateCalculator - INFO - Duplicate ratios has been calculated.
07/12/2022 05:28:42 - PrivacyScorer - INFO - The ratio of rows that were the same in both data has been calculated.
07/12/2022 05:28:42 - PrivacyScorer - INFO - The ratio of rows that were the same in both data has been calculated.
07/12/2022 05:28:42 - PrivacyScorer - INFO - Duplicate score has been calculated.


...

amatis/33float/analysis.log


07/12/2022 05:28:45 - SimilarityScorer - INFO - Similarity scoring for 1 combinations has been completed.
07/12/2022 05:28:45 - SimilarityScorer - INFO - Similarity scoring for 2 combinations has been started.
07/12/2022 05:29:01 - SimilarityScorer - INFO - Similarity scoring for 2 combinations has been completed.
07/12/2022 05:29:03 - SimilarityScorer - INFO - SIMILARITY CALCULATIONS HAS BEEN COMPLETED.

07/12/2022 05:29:03 - NumericalCredibility - INFO - NUMERICAL CREDIBILITY CALCULATIONS HAS BEEN STARTED.

07/12/2022 05:29:03 - NumericalCredibility - INFO - Consistency scoring has been started for CROSSING.
07/12/2022 05:29:03 - NumericalCredibility - INFO - Consistency scoring has been calculated for CROSSING.
07/12/2022 05:29:03 - NumericalCredibility - INFO - Consistency scoring has been started for FINISSHING.
07/12/2022 05:29:03 - NumericalCredibility - INFO - Consistency scoring has been calculated for FINISSHING.
07/12/2022 05:29:03 - NumericalCredibility - INFO - Consistency

amatis/33float/analysis.log


07/12/2022 05:29:03 - NumericalCredibility - INFO - Consistency scoring has been calculated for GK_HANDLING.
07/12/2022 05:29:03 - NumericalCredibility - INFO - Consistency scoring has been started for GK_KICKING.
07/12/2022 05:29:03 - NumericalCredibility - INFO - Consistency scoring has been calculated for GK_KICKING.
07/12/2022 05:29:03 - NumericalCredibility - INFO - Consistency scoring has been started for GK_POSITIONING.
07/12/2022 05:29:03 - NumericalCredibility - INFO - Consistency scoring has been calculated for GK_POSITIONING.
07/12/2022 05:29:03 - NumericalCredibility - INFO - Consistency scoring has been started for GK_REFLEXES.
07/12/2022 05:29:03 - NumericalCredibility - INFO - Consistency scoring has been calculated for GK_REFLEXES.
07/12/2022 05:29:03 - NumericalCredibility - INFO - All consistency scores has been calculated.
07/12/2022 05:29:03 - NumericalCredibility - INFO - Correlation scoring has been started.
07/12/2022 05:29:03 - NumericalCredibility - INFO - All 

MODULE D ->  SUCCESS True


In [11]:
result_status, result_message = generate_report(file_path)
print("MODULE E -> ", result_message, result_status)

[NbConvertApp] Converting notebook ReportVisualization.ipynb to HTML


MODULE E ->  SUCCESS True


[NbConvertApp] Writing 6232309 bytes to ReportVisualization.html
